<br></br>
# Data Mining and Decision Systems ACW
<br></br>
#### Student number: 201601628
<br>
<hr>

<a id="0"></a>
## Contents
<br>
<div style="line-height:2;">
<div style="line-height:2.5;"><a href="#4">....<h3 style="display: inline"> 0. Notebook Initialisation</h3></a></div>
<div style="line-height:2.5;"><a href="#1">....<h3 style="display: inline"> 1. Methodology</h3></a></div>
<div><a href="#11">........<h4 style="display: inline"> 1.1. CRISP DM</h4></a></div>
<div><a href="#111">............<h5 style="display: inline"> 1.1.1. Business Understanding</h5></a></div>
<div><a href="#112">............<h5 style="display: inline"> 1.1.2. Data Understanding</h5></a></div>
<div><a href="#113">............<h5 style="display: inline"> 1.1.3. Data Preperation</h5></a></div>
<div><a href="#114">............<h5 style="display: inline"> 1.1.4. Modeling</h5></a></div>
<div><a href="#115">............<h5 style="display: inline"> 1.1.5. Evaluation</h5></a></div>
<div style="line-height:2.5;"><a href="#116">............<h5 style="display: inline"> 1.1.6. Deployment</h5></a></div>
<div style="line-height:2.5;"><a href="#2">....<h3 style="display: inline">2. Results</h3></a></div>
<div style="line-height:2.5;"><a href="#3">....<h3 style="display: inline">3. Evaluation & Discussion</h3></a></div>
</div>
<br>
<hr>

<a id="4"></a>
## 0. Notebook Initialisation

In [6]:
## Import all libraries for use in notebook.
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np

from sklearn import metrics, model_selection, linear_model
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

%matplotlib inline

In [41]:
# Load the data and get some basic information for future use.
path = "data.csv" ## Relative path to train/test data.
rawData = pd.read_csv(path) ## Will reamin untouched for reference.
rawData.columns = [col.lower() for col in rawData.columns] ## Make headers lowercase to avoid some trivial errors.
df = rawData ## Working copy.

rawNRows = rawData.shape[0]
rawNCols = rawData.shape[1]
rawColNames = [colName.lower() for colName in rawData.columns.values] ## Convert colnames to lower for checking to avoid trivial errors.

rawData ## Show

,random,id,indication,diabetes,ihd,hypertension,arrhythmia,history,ipsi,contra,label
0,0.602437,218242,A-F,no,no,yes,no,no,78.0,20,NoRisk
1,0.602437,159284,TIA,no,no,no,no,no,70.0,60,NoRisk
2,0.602437,106066,A-F,no,yes,yes,no,no,95.0,40,Risk
3,0.128157,229592,TIA,no,no,yes,no,no,90.0,85,Risk
4,0.676862,245829,CVA,no,no,no,no,no,70.0,20,NoRisk
...,...,...,...,...,...,...,...,...,...,...,...
1515,0.391440,93406,A-F,no,yes,no,no,no,76.0,60,NoRisk
1516,0.253504,121814,A-F,no,no,yes,yes,no,90.0,75,Risk
1517,0.620373,101754,TIA,no,no,yes,no,no,75.0,20,NoRisk
1518,0.639342,263836,A-F,no,yes,no,no,no,70.0,45,NoRisk


<a href="#0">[back to top]</a>

<a id="1"></a>
## 1. Methodology
<i style="font-size: 14px">Provide details on the methodology applied towards the data mining analysis undertaken, providing rationale for these steps.

This should detail how you went from the raw data provided to the chosen model(s), choice of model, and how this methodology helps address the problem domain.

Evidence to support the following of this methodology should be presented, especially any cases which required moving backwards in the process to readdress issues.
</i>

<a href="#0">[back to top]</a>

<a id="11"></a>
### &nbsp;&nbsp;&nbsp;&nbsp;1.1. CRISP DM
<img src="crisp-dm.png" style="max-height:400px">
<a href="#0">[back to top]</a>

<a id="111"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.1. Business Understanding
<i style="font-size: 14px">Focuses on understanding the project objectives and requirements from a business perspective, and then converting this knowledge into a data mining problem definition and a preliminary plan.</i>

<a href="#0">[back to top]</a>

##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Problem Description
<div style="font-size: 14px">
<p>DOMAIN: Cardio-vascular medicine / healthcare</p>
<p>TYPE: Classification</p>
<p>INPUTS: Tabulated patient data</p>
<p>OUTPUTS:</p>
    <ul>
        <li>Risk</li>
        <li>No Risk</li>
    </ul>
<p>TERMINOLOGY/NUANCES:</p>
    <ul>
        <li>Risk</li>
        <li>No Risk</li>
    </ul>
</div>

<a id="112"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.2. Data Understanding
<i style="font-size: 14px">Starts with an initial data collection and proceeds with activities in order to get familiar with the data, to identify data quality problems, to discover first insights into the data, or to detect interesting subsets to form hypotheses for hidden information.</i>

<a href="#0">[back to top]</a>

##### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Data Dictionary

<table>
    <tbody>
        <tr>
            <td>
                <p><strong>Attribute</strong></p>
            </td>
            <td>
                <p><strong>Value Type</strong></p>
            </td>
            <td>
                <p><strong>NumberOfValues</strong></p>
            </td>
            <td>
                <p><strong>Values</strong></p>
            </td>
            <td>
                <p><strong>Comment</strong></p>
            </td>
            <td>
                <p><strong>Non-clinical Description</strong></p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Random</p>
            </td>
            <td>
                <p>Real</p>
            </td>
            <td>
                <p>Number of Records</p>
            </td>
            <td>
                <p>Unique</p>
            </td>
            <td>
                <p>Real number of help in randomly sorting the data records</p>
            </td>
            <td>
                <p>Real number of&nbsp;help&nbsp;in randomly sorting the data records: Should be unique values.</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Id</p>
            </td>
            <td>
                <p>Integer</p>
            </td>
            <td>
                <p>Max of Number of Records</p>
            </td>
            <td>
                <p>Unique to patient</p>
            </td>
            <td>
                <p>Anonymous patient record identifier: Should be unique values unless patient has multiple sessions</p>
            </td>
            <td>
                <p>Anonymous patient record identifier: Should be unique value per patient. Patient can have multiple sessions</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Indication</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Four</p>
            </td>
            <td>
                <p>{a-f, asx, cva, tia}</p>
            </td>
            <td>
                <p>What type of Cardiovascular event triggered the hospitalisation?</p>
            </td>
            <td>
                <p>What type of Cardiovascular event triggered the hospitalisation?</p><p> a-f :&nbsp;Atrial-Fibrillation</p>
                <p>asx&nbsp;:&nbsp;Asymptomatic Stenosis&nbsp;</p><p>cva&nbsp;: Cardiovascular Arrest</p>
                <p>tia&nbsp;:&nbsp;Transient Ischemic Attack ("mini-heart attack")</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Diabetes</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{no, yes}</p>
            </td>
            <td>
                <p>Does the patient suffer from Diabetes?</p>
            </td>
            <td>
                <p>Does the patient suffer from Diabetes?</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>IHD</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{no, yes}</p>
            </td>
            <td>
                <p>Does the patient suffer from Coronary artery disease (CAD), also known as ischemic heart disease (IHD)?</p>
            </td>
            <td>
                <p>Does the patient suffer from Coronary artery disease (CAD), also known as ischemic heart disease (IHD)?</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Hypertension</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{no, yes}</p>
            </td>
            <td>
                <p>Does the patient suffer from Hypertension?</p>
            </td>
            <td>
                <p>Does the patient suffer from Hypertension?</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Arrhythmia</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{no, yes}</p>
            </td>
            <td>
                <p>Does the patient suffer from</p>
                <p>Arrhythmia (i.e. erratic heart beat)?</p>
            </td>
            <td>
                <p>Does the patient suffer from Arrhythmia (i.e. erratic&nbsp;heart beat)?</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>History</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{no, yes}</p>
            </td>
            <td>
                <p>Has the patient a history of</p>
                <p>Cardiovascular interventions?</p>
            </td>
            <td>
                <p>Has the patient a history of Cardiovascular interventions?</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>IPSI</p>
            </td>
            <td>
                <p>Integer</p>
            </td>
            <td>
                <p>Potentially 101</p>
            </td>
            <td>
                <p>[0, 100]</p>
            </td>
            <td>
                <p>Percentage figure for cerebral ischemic lesions defined as ipsilateral</p>
            </td>
            <td>
                <p>Percentage figure for cerebral ischemic lesions defined as ipsilateral</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Contra</p>
            </td>
            <td>
                <p>Integer</p>
            </td>
            <td>
                <p>Potentially 101</p>
            </td>
            <td>
                <p>[0, 100]</p>
            </td>
            <td>
                <p>Percentage figure for contralateral cerebral ischemic lesions</p>
            </td>
            <td>
                <p>Percentage figure for contralateral cerebral ischemic lesions</p>
            </td>
        </tr>
        <tr>
            <td>
                <p>Label</p>
            </td>
            <td>
                <p>Nominal</p>
            </td>
            <td>
                <p>Two</p>
            </td>
            <td>
                <p>{risk, norisk}</p>
            </td>
            <td>
                <p>Is the patient at risk (Mortality)?</p>
            </td>
            <td>
                <p>Is the patient at risk (Mortality)?</p>
            </td>
        </tr>
    </tbody>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b style="color: red;">NOTE:</b>
<p style="font-size: 14px">"Session" is also included in the non-clinical description, but not included in the data dictionary.</p>
<br>
<table>
    <tr>
        <td>
            <p><strong>Attribute</strong></p>
        </td>
        <td>
            <p><strong>Value Type</strong></p>
        </td>
        <td>
            <p><strong>NumberOfValues</strong></p>
        </td>
        <td>
            <p><strong>Values</strong></p>
        </td>
        <td>
            <p><strong>Comment</strong></p>
        </td>
        <td>
            <p><strong>Non-clinical Description</strong></p>
        </td>
    </tr>
    <tr>
        <td>
            <p>Session</p>
        </td>
        <td>
            <p>Unknown</p>
        </td>
        <td>
            <p>Max Number of Records (assumed)</p>
        </td>
        <td>
            <p>Unique to patient</p>
        </td>
        <td>
            <p>Unknown</p>
        </td>
        <td>
            <p>Anonymous patient session identifier.</p>
        </td>
    </tr>
</table>

<p><b>Step 1. </b>Check data for conformity to dictionary.</p>

(a) Define a set of rules/assumptions as provided in the dictionary.

In [42]:
## Note: data type and number of vals can be inferred where vals are given.
dataAssumptions = {
    "random":{ ## Col name.
        "valType": float, ## Real.
        "nValues": (nRows, nRows), # Unique per record.
    },  
    "id":{
        "valType": int,## Integer.
        "nValues": (1, nRows), ## Unique per patient.
    },
    "indication":{
        "vals": ["a-f","asx","cva","tia"] ## Possible values (except nan).
    },
    "diabetes":{
        "vals": ["yes", "no"]
    },
    "ihd":{
        "vals": ["yes", "no"]
    },
    "hypertension":{
        "vals": ["yes", "no"]
    },
    "arrhythmia":{
        "vals": ["yes", "no"]
    },
    "history": {
        "vals": ["yes", "no"]
    },
    "ipsi": {
        "vals": np.arange(0,101)
    },
    "contra": {
        "vals": np.arange(0,101),
    },
    "label": {
        "vals": ["risk", "norisk"]
    },
    "session":{
        "valType": object, # Unknown.
        "nValues": (1, nRows), ## Unique per patient (assumed).
    },
}

(b) Compare assumptions with actual (raw) data.

In [43]:
## Check for all expected columns exist in the data.
missingCols = []
for k, v in dataAssumptions.items(): ## k = key, v == value.
    if (k not in rawColNames):
        missingCols.append(k)
if (len(missingCols) > 0):
    print("The following expected columns are missing from the data:")
    print(missingCols)

The following expected columns are missing from the data:
['session']


In [71]:
## Check number of values of each column against expected.
concerns = []
for k, v in dataAssumptions.items(): ## k = key, v == value.
    if (k in rawColNames):
        actualNValues = len(rawData[k].dropna().unique()) ## Ignore nan values for this check.
        try:
            expectedNValues = (1, len(v["vals"])) 
        except:
            expectedNValues = v["nValues"]

        if (not(actualNValues >= expectedNValues[0]) or not(actualNValues <= expectedNValues[1])):            
            concerns.append(("Col: " + str(k), "Expected: " + str(expectedNValues), "Actual: " + str(actualNValues)))
        
if (len(concerns) > 0):
    print("The following columns have unexpected values:")
    for concern in concerns:
        print(concern)

The following columns have unexpected values:
('Col: random', 'Expected: (1520, 1520)', 'Actual: 1222')
('Col: indication', 'Expected: (1, 4)', 'Actual: 5')
('Col: label', 'Expected: (1, 2)', 'Actual: 3')


In [ ]:
## Check actual values of each column against expected.
concerns = []
for k, v in dataAssumptions.items(): ## k = key, v == value.
    if (k in rawColNames):
        actualNValues = len(rawData[k].dropna().unique()) ## Ignore nan values for this check.
        try:
            expectedNValues = v["vals"]) 
        except:
            expectedNValues = v["nValues"]

        if (not(actualNValues >= expectedNValues[0]) or not(actualNValues <= expectedNValues[1])):            
            concerns.append(("Col: " + str(k), "Expected: " + str(expectedNValues), "Actual: " + str(actualNValues)))
        
if (len(concerns) > 0):
    print("The following columns have unexpected values:")
    for concern in concerns:
        print(concern)

<a id="113"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.3. Data Preperation
<i style="font-size: 14px">The data preparation phase covers all activities to construct the final dataset from the initial raw data.</i>

<a href="#0">[back to top]</a>

<a id="114"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.4. Modeling
<i style="font-size: 14px">Modeling techniques are selected and applied.  Since some techniques like neural nets have specific requirements regarding the form of the data, there can be a loop back here to data prep.</i>

<a href="#0">[back to top]</a>

<a id="115"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.5. Evaluation
<i style="font-size: 14px">Once one or more models have been built that appear to have high quality based on whichever loss functions have been selected, these need to be tested to ensure they generalize against unseen data and that all key business issues have been sufficiently considered.  The end result is the selection of the champion model(s).</i>

<a href="#0">[back to top]</a>

<a id="116"></a>
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1.5. Deployment
<i style="font-size: 14px">Generally this will mean deploying a code representation of the model into an operating system to score or categorize new unseen data as it arises and to create a mechanism for the use of that new information in the solution of the original business problem.  Importantly, the code representation must also include all the data prep steps leading up to modeling so that the model will treat new raw data in the same manner as during model development.

You may well observe that there is nothing special here and that’s largely true.  From today’s data science perspective this seems like common sense.  This is exactly the point.  The common process is so logical that it has become embedded into all our education, training, and practice.</i>

<a href="#0">[back to top]</a>

<a id="2"></a>
## &nbsp;&nbsp;&nbsp;&nbsp;2. Results
<i style="font-size: 14px">Results should include tables showing model performance with appropriately selected metrics. No rationale should be provided for this section - simply results of evaluative processes.

If using modified variants of the dataset, these should be clearly identified in the tables with appropriate naming. The justification and description of modification is not for this section.

Additional figures may be used as appropriate, in support of discussion points in the Evaluation & Discussion section, or as evidence for methodology following above.</i>

<a href="#0">[back to top]</a>

<a id="3"></a>
## &nbsp;&nbsp;&nbsp;&nbsp;3. Evaluation & Discussion
<i style="font-size: 14px">Evaluation methodology used for generating the results provided in the previous section. How were these evaluated? Why was this selected? What metrics were used and why?

Discussion of the results should be presented with appropriate evidence and rationale. E.g Which is the best model, and why? 

Consider each stage in the methodology, and reflect on any improvements which could have been made. Could any techniques have been used which may have improved performance? Why?</i>

<a href="#0">[back to top]</a>